##### target variable = `BM_GERMANY_POWER_M1_CLOSE_EUR_MWH`

In [24]:
import pandas as pd 
import numpy as np 
import statsmodels.api as sm 
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 

In [25]:
#read in data
df = pd.read_csv('processed_data.csv')

In [26]:
# setting the date as index and sorting it 
df["Date"] = pd.to_datetime(df["Date"].astype(int), origin="1899-12-30", unit="D")
df = df.set_index("Date").sort_index()

In [27]:
df.head()

,BM_TTF_M1_OPEN_EUR_MWH,BM_TTF_M1_HIGH_EUR_MWH,BM_TTF_M1_LOW_EUR_MWH,BM_TTF_M1_CLOSE_EUR_MWH,BM_TTF_M1_VOLUME_MWH,BM_TTF_M1_OPEN_EUR_MWH.1,BM_TTF_M1_HIGH_EUR_MWH.1,BM_TTF_M1_LOW_EUR_MWH.1,BM_TTF_M1_CLOSE_EUR_MWH.1,BM_TTF_M1_VOLUME_MWH.1,...,MF_LOAD_FACTOR_GERMANY_PV_LOAD_FACTOR_PCT,MF_LOAD_FACTOR_GERMANY_WIND_LOAD_FACTOR_PCT,MF_NUCLEAR_GENERATION_FRANCE_GENERATION_GW,MF_NORWAY_GAS_OUTAGES_ACTUAL_VALUE_GWH_D,MF_NORWAY_GAS_OUTAGES_SCHEDULED_M1_VALUE_GWH_D,MF_NORWAY_GAS_IMPORT_ACTUAL_VALUE_GWH_D,MF_UK_BE_INTERCONNECTOR_FLOW_ACTUAL_FLOW_GWH_D,MF_RUSSIAN_PIPELINE_FLOW_ACTUAL_FLOW_GWH_D,MF_LNG_EUROPE_FLOW_ACTUAL_FLOW_GWH_D,MF_EU_STORAGE_STORAGE_ACTUAL_STORAGE_TWH
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-05,18.75,17.93,19.00,17.88,23136,15.80,15.67,15.80,15.65,96360,...,0.003580,0.180808,51.169917,0.0,8.693452,3787.667023,-491.6088,2016.305299,1911.650,802.6733
2021-01-06,18.23,17.58,18.48,17.33,12384,15.67,15.59,15.67,15.50,490560,...,0.003343,0.099889,51.521500,0.0,8.693452,3755.702824,-522.2620,2030.335592,1979.025,793.5911
2021-01-07,18.38,19.40,19.45,18.38,24864,15.93,16.08,16.08,15.90,972360,...,0.003904,0.057446,51.705583,0.0,8.693452,3705.631693,-296.0362,2003.686192,2216.050,783.2034
2021-01-08,20.45,20.38,20.65,19.93,17808,16.40,16.35,16.40,16.25,385440,...,0.004496,0.062074,51.684875,0.0,8.693452,3763.715746,-304.6786,1964.979103,1965.025,771.0235
2021-01-11,21.50,22.55,22.60,21.35,31056,16.43,16.45,16.48,16.40,367920,...,0.012123,0.281651,51.711417,0.0,8.693452,3852.709043,-331.8976,1936.356517,1530.775,743.2713


In [29]:
target = "BM_GERMANY_POWER_M1_CLOSE_EUR_MWH"

#dropping other OHLCV of same contract and duplicate (.1) data
drop_cols = [c for c in df.columns if 'GERMANY_POWER_M1' in c and c!= target]
drop_cols.extend(
    c for c in df.columns if c.endswith(".1")
)
drop_cols = list(set(drop_cols))

df = df.drop(columns=drop_cols)
df.columns

Index(['BM_TTF_M1_OPEN_EUR_MWH', 'BM_TTF_M1_HIGH_EUR_MWH',
       'BM_TTF_M1_LOW_EUR_MWH', 'BM_TTF_M1_CLOSE_EUR_MWH',
       'BM_TTF_M1_VOLUME_MWH', 'BM_GERMANY_POWER_M1_CLOSE_EUR_MWH',
       'BM_GERMANY_POWER_CAL1_OPEN_EUR_MWH',
       'BM_GERMANY_POWER_CAL1_HIGH_EUR_MWH',
       'BM_GERMANY_POWER_CAL1_LOW_EUR_MWH',
       'BM_GERMANY_POWER_CAL1_CLOSE_EUR_MWH',
       'BM_GERMANY_POWER_CAL1_VOLUME_MWH', 'BM_EUA_CO2_CAL1_PRICE_EUR_TON',
       'IM_SPOT_GAS_PSV_DA_PRICE_EUR_MWH', 'IM_SPOT_GAS_TTF_DA_PRICE_EUR_MWH',
       'IM_COAL_GAS_SWITCH_M1_SUPPORT_EUR_MWH',
       'IM_COAL_GAS_SWITCH_M1_RESISTANCE_EUR_MWH',
       'IM_COAL_GAS_SWITCH_CAL1_SUPPORT_EUR_MWH',
       'IM_COAL_GAS_SWITCH_CAL1_RESISTANCE_EUR_MWH',
       'IM_TTF_OPTIONS_TTF_PREMIUM_EUR_MWH',
       'IM_TTF_OPTIONS_TTF_IMPLIED_VOL_PCT', 'IM_COAL_CAL1_PRICE_USD_TON',
       'IM_BRENT_M1_PRICE_USD_BBL', 'IM_JKM_LNG_M1_PRICE_USD_MMBTU',
       'IM_HENRY_HUB_SPOT_PRICE_USD_MMBTU', 'MF_COT_TTF_HEDGE_FUNDS_LONG_MWH',
       'M

In [30]:
# Separating features and target
X = df.drop(columns=[target])
y = df[target]

In [31]:
#dropping rows where target is missing 
mask = y.notna()
X = X.loc[mask]
y = y[mask]

In [ ]:
# Checking how much data is missing in each feature column
missing_pct = X.isnull().mean().sort_values(ascending=False)
print(missing_pct.head(20))

MF_COT_EUA_COMMERCIAL_SHORT_TON                                 0.00098
MF_COT_TTF_COMMERCIAL_RISK_REDUCING_NET_MWH                     0.00098
MF_COT_TTF_INVESTMENT_FIRMS_OR_CREDIT_INSTITUTIONS_NET_MWH      0.00098
MF_COT_TTF_OTHER_FINANCIAL_INSTITUTIONS_LONG_MWH                0.00098
MF_COT_TTF_OTHER_FINANCIAL_INSTITUTIONS_SHORT_MWH               0.00098
MF_COT_TTF_OTHER_FINANCIAL_INSTITUTIONS_NET_MWH                 0.00098
MF_COT_TTF_TOTAL_LONG_MWH                                       0.00098
MF_COT_TTF_TOTAL_SHORT_MWH                                      0.00098
MF_COT_TTF_TOTAL_NET_MWH                                        0.00098
MF_COT_EUA_HEDGE_FUNDS_LONG_TON                                 0.00098
MF_COT_EUA_HEDGE_FUNDS_SHORT_TON                                0.00098
MF_COT_EUA_HEDGE_FUNDS_NET_TON                                  0.00098
MF_COT_EUA_COMMERCIAL_LONG_TON                                  0.00098
MF_COT_EUA_COMMERCIAL_NET_TON                                   

###### ^ minimal missing

In [ ]:
#forward fill missing values 
# going top to bottom and replacing any missing values with the last non-missing value
X = X.ffill().bfill()

In [35]:
# dropping columns that have no variation (aka constant) since they won't affect the model
X = X.loc[:, X.std() > 1e-10]

In [ ]:
print(f"Final dataset: {X.shape[0]} obs × {X.shape[1]} features")
print(f"Target range: {y.min()} — {y.max()}, mean: {y.mean()}")
feature_names = X.columns.tolist()

Final dataset: 1020 obs × 90 features
Target range: 41.5 — 604.0, mean: 139.85833333333335


#### OLS REGRESSION

In [38]:
scaler = StandardScaler() # to normalize each feature
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=feature_names, index=X.index) #computing mean and std of every column and then transforming the value; after this every feature has mean 0 and std 1 
# now the coefs can reflect the effecr of a one std change in each feature making them comparable to one another 

X_OLS = sm.add_constant(X_scaled) # adding a constaant to get a y-intercept 
model = sm.OLS(y.values, X_OLS.values).fit()
# setting up the ordinary least squares regression and solving it -- returns regression table  

In [41]:
#coef table
coef_df = pd.DataFrame({
    "Variable": ["const"] + feature_names,
    "Coef": model.params, # estimated regression coefficients
    "Std_Err": model.bse, # standard errors of the coefficients
    "t_stat": model.tvalues, # t-statistics for each coefficient
    "p_value": model.pvalues, # p-values for each coefficient
})

# adding significance stars based on p-values
coef_df["Sig"] = coef_df["p_value"].apply(
    lambda p: "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""
)

# sorting by p-value
coef_df = coef_df.sort_values("p_value")
# lowest p-values = highest significane


print(f"\nR²:       {model.rsquared:.4f}") # amount of variance explained by the model
print(f"Adj R²:   {model.rsquared_adj:.4f}") 
print(f"F-stat:   {model.fvalue:.2f}  (p={model.f_pvalue:.2e})") #whether the model as a whole is statistically significant
print(f"AIC:      {model.aic:.1f}") # lower is better
print(f"BIC:      {model.bic:.1f}") # lower is better
print(f"DW stat:  {sm.stats.stattools.durbin_watson(model.resid):.3f}") # autocorrelation of residuals (2.0 = no autocorrelation, 0-2 = positive autocorrelation, 2-4 = negative autocorrelation)
print(f"Obs:      {model.nobs:.0f}") # number of observations used in the model
print()
print("Coefficients (sorted by significance) ")
print(coef_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))


R²:       0.9926
Adj R²:   0.9919
F-stat:   1460.91  (p=0.00e+00)
AIC:      7435.6
BIC:      7864.3
DW stat:  0.818
Obs:      1020

Coefficients (sorted by significance) 
                                                    Variable            Coef          Std_Err   t_stat  p_value Sig
                                                       const        139.8583           0.2785 502.2722   0.0000 ***
                                     BM_TTF_M1_CLOSE_EUR_MWH         80.1148           7.3156  10.9513   0.0000 ***
                            IM_SPOT_GAS_TTF_DA_PRICE_EUR_MWH        -27.8511           3.0886  -9.0174   0.0000 ***
                               BM_EUA_CO2_CAL1_PRICE_EUR_TON          6.8119           1.2889   5.2852   0.0000 ***
                 MF_WATER_RESERVOIR_NORMAL_ALPINE_NORMAL_TWH        -21.9065           4.3573  -5.0276   0.0000 ***
                            IM_SPOT_GAS_PSV_DA_PRICE_EUR_MWH         16.2934           3.3562   4.8548   0.0000 ***
                